In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import gc
import optuna
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
import warnings
import joblib
import json

# Suppress warnings
warnings.filterwarnings("ignore")

In [2]:
def load_data_with_scaling_encoding_dtypes():
    # Define dtypes for train_transaction
    dtypes_train_transaction = {
        'TransactionID': 'int32',
        'isFraud': 'int8',
        'TransactionDT': 'int32',
        'TransactionAmt': 'float32',
        'ProductCD': 'object',
        'card1': 'int16',
        'card2': 'float32',
        'card3': 'float32',
        'card4': 'object',
        'card5': 'float32',
        'card6': 'object',
        'addr1': 'float32',
        'addr2': 'float32',
        'dist1': 'float32',
        'dist2': 'float32',
        'P_emaildomain': 'object',
        'R_emaildomain': 'object',
        # Add all C, D, M, V features
        **{f'C{i}': 'float32' for i in range(1, 15)},
        **{f'D{i}': 'float32' for i in range(1, 16)},
        **{f'M{i}': 'object' for i in range(1, 10)},
        **{f'V{i}': 'float32' for i in range(1, 340)}
    }

    # Define dtypes for train_identity
    dtypes_train_identity = {
        **{f'id_{i:02}': 'float32' for i in range(1, 12)},
        'id_12': 'object',
        'id_13': 'float32',
        'id_14': 'float32',
        'id_15': 'object',
        'id_16': 'object',
        **{f'id_{i:02}': 'float32' for i in range(17, 40)},
        **{f'id_{i:02}': 'object' for i in range(23, 39)},
        'DeviceType': 'object',
        'DeviceInfo': 'object'
    }

    # Define dtypes for test datasets
    dtypes_test_transaction = dtypes_train_transaction.copy()
    del dtypes_test_transaction['isFraud']  # Test data does not contain 'isFraud'

    dtypes_test_identity = dtypes_train_identity.copy()

    # Read in datasets with dtypes
    train_transaction = pd.read_csv(
        '/kaggle/input/ieee-fraud-detection/train_transaction.csv', 
        dtype=dtypes_train_transaction
    )
    train_identity = pd.read_csv(
        '/kaggle/input/ieee-fraud-detection/train_identity.csv', 
        dtype=dtypes_train_identity
    )
    test_transaction = pd.read_csv(
        '/kaggle/input/ieee-fraud-detection/test_transaction.csv', 
        dtype=dtypes_test_transaction
    )
    test_identity = pd.read_csv(
        '/kaggle/input/ieee-fraud-detection/test_identity.csv', 
        dtype=dtypes_test_identity
    )

    # Standardize column names
    test_identity.columns = test_identity.columns.str.replace('-', '_')
    test_transaction.columns = test_transaction.columns.str.replace('-', '_')

    # Merge datasets
    train = train_transaction.merge(train_identity, how='left', on='TransactionID')
    test = test_transaction.merge(test_identity, how='left', on='TransactionID')

    # Free up memory
    del train_transaction, train_identity, test_transaction, test_identity
    gc.collect()

    # Handle missing values
    train.fillna(-999, inplace=True)
    test.fillna(-999, inplace=True)

    # Define categorical features
    categorical_features = [
        'ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain',
        'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
        'id_33', 'id_34', 'DeviceType', 'DeviceInfo'
    ]
    categorical_features += [f'id_{i}' for i in range(12, 39)]

    # Encode categorical features
    for col in categorical_features:
        if col in train.columns:
            train[col] = train[col].astype(str)
            test[col] = test[col].astype(str)

            le = LabelEncoder()
            combined_data = pd.concat([train[col], test[col]], axis=0)
            le.fit(combined_data)
            train[col] = le.transform(train[col])
            test[col] = le.transform(test[col])

    return train, test

# Load data
train, test = load_data_with_scaling_encoding_dtypes()


In [3]:
X = train.drop(columns=['isFraud', 'TransactionID'])
y = train['isFraud']
X_test = test.drop(columns=['TransactionID'], errors='ignore')

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [4]:
def optuna_xgb(trial):
    print(f"[XGBoost][Trial {trial.number}] Starting optimization with params:")
    
    # Define trial hyperparameters
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_float('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'lambda': trial.suggest_float('lambda', 0, 10),
        'alpha': trial.suggest_float('alpha', 0, 10),
        'n_estimators': 1000
    }
    
    print(f"[XGBoost][Trial {trial.number}] Parameters: {params}")
    
    # Initialize and train the XGBoost model
    model = XGBClassifier(**params, use_label_encoder=False)
    print(f"[XGBoost][Trial {trial.number}] Training started...")
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=100,
        verbose=100  # Print progress every 100 iterations
    )
    print(f"[XGBoost][Trial {trial.number}] Training completed.")
    
    # Predict probabilities and calculate AUC
    val_pred = model.predict_proba(X_val)[:, 1]
    auc_score = roc_auc_score(y_val, val_pred)
    print(f"[XGBoost][Trial {trial.number}] AUC Score: {auc_score:.4f}")
    
    return auc_score

# Run Optuna study
print("[XGBoost] Starting hyperparameter optimization with Optuna...")
study_name = "xgboost_optimization"
study_xgb = optuna.create_study(study_name=study_name, direction="maximize")
study_xgb.optimize(optuna_xgb, n_trials=50, n_jobs=1)

[I 2025-03-18 16:24:57,441] A new study created in memory with name: xgboost_optimization


[XGBoost] Starting hyperparameter optimization with Optuna...
[XGBoost][Trial 0] Starting optimization with params:
[XGBoost][Trial 0] Parameters: {'objective': 'binary:logistic', 'eval_metric': 'auc', 'learning_rate': 0.04193951129251835, 'max_depth': 10, 'min_child_weight': 5.684029759472064, 'gamma': 0.19586268417730812, 'subsample': 0.9820350309865852, 'colsample_bytree': 0.604597367901569, 'lambda': 9.985724603916392, 'alpha': 6.285876395469289, 'n_estimators': 1000}
[XGBoost][Trial 0] Training started...
[0]	validation_0-auc:0.83242
[100]	validation_0-auc:0.91641
[200]	validation_0-auc:0.93625
[300]	validation_0-auc:0.94400
[400]	validation_0-auc:0.94965
[500]	validation_0-auc:0.95389
[600]	validation_0-auc:0.95722
[700]	validation_0-auc:0.95969
[800]	validation_0-auc:0.96186
[900]	validation_0-auc:0.96340
[999]	validation_0-auc:0.96474
[XGBoost][Trial 0] Training completed.


[I 2025-03-18 16:36:26,784] Trial 0 finished with value: 0.9647428412182409 and parameters: {'learning_rate': 0.04193951129251835, 'max_depth': 10, 'min_child_weight': 5.684029759472064, 'gamma': 0.19586268417730812, 'subsample': 0.9820350309865852, 'colsample_bytree': 0.604597367901569, 'lambda': 9.985724603916392, 'alpha': 6.285876395469289}. Best is trial 0 with value: 0.9647428412182409.


[XGBoost][Trial 0] AUC Score: 0.9647
[XGBoost][Trial 1] Starting optimization with params:
[XGBoost][Trial 1] Parameters: {'objective': 'binary:logistic', 'eval_metric': 'auc', 'learning_rate': 0.01731619152566296, 'max_depth': 8, 'min_child_weight': 5.68112193955543, 'gamma': 0.43419181402534257, 'subsample': 0.8349235632540425, 'colsample_bytree': 0.9366084156559806, 'lambda': 0.2542918718348608, 'alpha': 4.211957755413984, 'n_estimators': 1000}
[XGBoost][Trial 1] Training started...
[0]	validation_0-auc:0.80625
[100]	validation_0-auc:0.88530
[200]	validation_0-auc:0.90695
[300]	validation_0-auc:0.92238
[400]	validation_0-auc:0.92958
[500]	validation_0-auc:0.93442
[600]	validation_0-auc:0.93799
[700]	validation_0-auc:0.94122
[800]	validation_0-auc:0.94407
[900]	validation_0-auc:0.94679
[999]	validation_0-auc:0.94915
[XGBoost][Trial 1] Training completed.


[I 2025-03-18 16:46:59,871] Trial 1 finished with value: 0.9491538617774102 and parameters: {'learning_rate': 0.01731619152566296, 'max_depth': 8, 'min_child_weight': 5.68112193955543, 'gamma': 0.43419181402534257, 'subsample': 0.8349235632540425, 'colsample_bytree': 0.9366084156559806, 'lambda': 0.2542918718348608, 'alpha': 4.211957755413984}. Best is trial 0 with value: 0.9647428412182409.


...

[XGBoost][Trial 47] AUC Score: 0.9692
[XGBoost][Trial 48] Starting optimization with params:
[XGBoost][Trial 48] Parameters: {'objective': 'binary:logistic', 'eval_metric': 'auc', 'learning_rate': 0.04978294457730683, 'max_depth': 15, 'min_child_weight': 5.028299322319862, 'gamma': 1.4359233147662676, 'subsample': 0.8417090098120228, 'colsample_bytree': 0.6624409118445584, 'lambda': 4.23372246764718, 'alpha': 0.5221731058205054, 'n_estimators': 1000}
[XGBoost][Trial 48] Training started...
[0]	validation_0-auc:0.86393
[100]	validation_0-auc:0.94530
[200]	validation_0-auc:0.96089
[300]	validation_0-auc:0.96611
[400]	validation_0-auc:0.96809
[500]	validation_0-auc:0.96892
[600]	validation_0-auc:0.96946
[700]	validation_0-auc:0.96980
[800]	validation_0-auc:0.97014
[900]	validation_0-auc:0.97034
[999]	validation_0-auc:0.97041
[XGBoost][Trial 48] Training completed.


[I 2025-03-19 01:36:29,749] Trial 48 finished with value: 0.9704074858615013 and parameters: {'learning_rate': 0.04978294457730683, 'max_depth': 15, 'min_child_weight': 5.028299322319862, 'gamma': 1.4359233147662676, 'subsample': 0.8417090098120228, 'colsample_bytree': 0.6624409118445584, 'lambda': 4.23372246764718, 'alpha': 0.5221731058205054}. Best is trial 27 with value: 0.9752082179571367.


[XGBoost][Trial 48] AUC Score: 0.9704
[XGBoost][Trial 49] Starting optimization with params:
[XGBoost][Trial 49] Parameters: {'objective': 'binary:logistic', 'eval_metric': 'auc', 'learning_rate': 0.03973907524592974, 'max_depth': 4, 'min_child_weight': 5.728881488837832, 'gamma': 4.2152414741696695, 'subsample': 0.9171219194849818, 'colsample_bytree': 0.6852835513443649, 'lambda': 2.4476604226961673, 'alpha': 1.4425539076424647, 'n_estimators': 1000}
[XGBoost][Trial 49] Training started...
[0]	validation_0-auc:0.70453
[100]	validation_0-auc:0.87913
[200]	validation_0-auc:0.89313
[300]	validation_0-auc:0.90014
[400]	validation_0-auc:0.90554
[500]	validation_0-auc:0.90896
[600]	validation_0-auc:0.91227
[700]	validation_0-auc:0.91504
[800]	validation_0-auc:0.91794
[900]	validation_0-auc:0.92018
[999]	validation_0-auc:0.92219
[XGBoost][Trial 49] Training completed.


[I 2025-03-19 01:45:38,081] Trial 49 finished with value: 0.9221925103067043 and parameters: {'learning_rate': 0.03973907524592974, 'max_depth': 4, 'min_child_weight': 5.728881488837832, 'gamma': 4.2152414741696695, 'subsample': 0.9171219194849818, 'colsample_bytree': 0.6852835513443649, 'lambda': 2.4476604226961673, 'alpha': 1.4425539076424647}. Best is trial 27 with value: 0.9752082179571367.


[XGBoost][Trial 49] AUC Score: 0.9222


In [5]:
best_xgb_params = study_xgb.best_params
print(f"[XGBoost] Best Hyperparameters: {best_xgb_params}")
with open("xgb_best_params.json", "w") as f:
    json.dump(best_xgb_params, f)

[XGBoost] Best Hyperparameters: {'learning_rate': 0.046102039811631704, 'max_depth': 15, 'min_child_weight': 4.251287229768353, 'gamma': 0.02979780767635686, 'subsample': 0.8370876067609905, 'colsample_bytree': 0.6398296464082214, 'lambda': 5.748245515801523, 'alpha': 0.9514047373597887}


In [6]:
optimized_xgb_model = XGBClassifier(**best_xgb_params, use_label_encoder=False)
optimized_xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=100,
    verbose=100
)

[0]	validation_0-logloss:0.20197
[99]	validation_0-logloss:0.06893


XGBClassifier(alpha=0.9514047373597887, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6398296464082214, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.02979780767635686,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=5.748245515801523,
              learning_rate=0.046102039811631704, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=4.251287229768353, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, ...)

In [7]:
joblib.dump(optimized_xgb_model, "optimized_xgb_model.pkl")

['optimized_xgb_model.pkl']

In [8]:
val_pred = optimized_xgb_model.predict_proba(X_val)[:, 1]
val_predictions = pd.DataFrame({'isFraud': y_val.values, 'predicted': val_pred})
val_predictions.to_csv("xgb_val_predictions.csv", index=False)

In [9]:
val_auc = roc_auc_score(y_val, val_pred)
print(f"Validation AUC: {val_auc:.4f}")

Validation AUC: 0.9418


In [10]:
test_predictions = optimized_xgb_model.predict_proba(X_test)[:, 1]

# Save test predictions for submission
submission = pd.DataFrame(
    {'TransactionID': test['TransactionID'], 'isFraud': test_predictions}
)
submission.to_csv("submission.csv", index=False)
submission.to_csv("xgb_test_predictions.csv", index=False)

In [11]:
classification_report(y_val,val_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets